In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplt
import matplotlib as plt
import datetime as dati

In [2]:
dataset = pd.read_csv("./Train_TP2_Datos_2020-2C.csv", parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                                                           'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                                                           'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
dataset = dataset.drop(columns= ['Prod_Category_A','Product_Category_B','Actual_Delivery_Date','Submitted_for_Approval','Last_Activity','Price','Currency','Brand','Size','Product_Type'])

dataset2 = pd.read_csv("./Test_TP2_Datos_2020-2C.csv", parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                                                           'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                                                           'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
#dataset = dataset.drop(columns= [Brand','Size','Product_Type'])
dataset2 = dataset2.drop(columns= ['Prod_Category_A','Product_Category_B','Actual_Delivery_Date','Submitted_for_Approval','Last_Activity','Price','Currency','Brand','Size','Product_Type'])

In [22]:
#dataset.columns.intersection(dataset2.columns)
#dataset2.columns.intersection(['Region', 'Territory','Bureaucratic_Code', 'Billing_Country',
#                              'Account_Name','Account_Owner', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms',
#               'Last_Modified_By', 'Product_Family','Product_Name', 'Month', 'Delivery_Quarter'])

dataset2.dtypes

ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

In [4]:
#limpio los datos no finalzizados
dataset = dataset.loc[(dataset['Stage']== 'Closed Won') | (dataset['Stage']== 'Closed Lost')]
dataset['Stage'].value_counts()

Closed Won     9533
Closed Lost    7350
Name: Stage, dtype: int64

In [5]:
#mergeo ambos DS con countes y ganados, en el caso de dataset 2 los cunters y los ganados son siempre 0 así no afectan los promedios.
dataset['ganado']= (dataset['Stage']== 'Closed Won' )
dataset.drop(columns= ['Stage'])
dataset['counter']= 1

dataset2['ganado']= 0
dataset2['counter']= 0

dataset_merge= dataset.append(dataset2).sort_values('Opportunity_Created_Date').reset_index().drop(columns=['index'])
#dataset_merge['counter'].value_counts()
dataset_merge.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Quarter,Delivery_Year,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,ganado,counter
0,12714,EMEA,France,0,0,0,0,Bureaucratic_Code_4,2013-07-27,None,...,Q4,2018,2,EUR,662287.5,EUR,662287.5,Closed Lost,0,1
1,18684,APAC,Philippines,1,1,1,1,Bureaucratic_Code_3,2014-01-22,Source_3,...,Q2,2016,20,USD,9803430.0,USD,9803430.0,Closed Lost,0,1
2,20675,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,Q3,2016,0,USD,170931.6,USD,341863.2,Closed Lost,0,1
3,20682,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,Q3,2016,0,USD,170931.6,USD,341863.2,Closed Lost,0,1
4,20513,Americas,None,1,0,1,0,Bureaucratic_Code_5,2013-08-22,Source_7,...,Q1,2016,6,USD,4395304.0,USD,31410780.0,Closed Won,1,1


In [6]:
categorical = ['Region', 'Territory', 'Product_Name', 'Account_Owner','Bureaucratic_Code','Billing_Country', 
               'Account_Name', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms', 'Product_Family', 
               'Month', 'Delivery_Quarter']
for agrupador in categorical:
    gruped_product = dataset_merge.groupby(agrupador)[['ganado','counter']].cumsum()
    gruped_product[agrupador + '_mean']= np.NaN
    gruped_product.loc[(gruped_product['counter'] != 0), agrupador + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
    gruped_product= gruped_product.rename(columns = {'ganado':(agrupador + '_ganados'), 'counter': (agrupador + '_count') })
    dataset_merge= dataset_merge.join(gruped_product)
    print(gruped_product.shape)

#agrupador = 'Opportunity_Owner' 
#gruped_product = dataset_merge.groupby(agrupador)[['ganado','counter']].cumsum()
#gruped_product[agrupador + '_mean']= np.NaN
#gruped_product.loc[(gruped_product['counter'] != 0), agrupador + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
#gruped_product= gruped_product.rename(columns = {'ganado':(agrupador + '_ganados'), 'counter': (agrupador + '_count') })

#dataset_merge= dataset_merge.join(gruped_product)


(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)


In [8]:
dataset_merge.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Terms_mean,Product_Family_ganados,Product_Family_count,Product_Family_mean,Month_ganados,Month_count,Month_mean,Delivery_Quarter_ganados,Delivery_Quarter_count,Delivery_Quarter_mean
0,12714,EMEA,France,0,0,0,0,Bureaucratic_Code_4,2013-07-27,None,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
1,18684,APAC,Philippines,1,1,1,1,Bureaucratic_Code_3,2014-01-22,Source_3,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
2,20675,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
3,20682,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,0.000000,0,2,0.0,0,1,0.0,0,2,0.0
4,20513,Americas,None,1,0,1,0,Bureaucratic_Code_5,2013-08-22,Source_7,...,0.333333,1,1,1.0,1,1,1.0,1,1,1.0


In [9]:
dataset.dtypes


ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

# categorical de a pares

In [10]:
for i in range(len(categorical) -1):
    for j in range(i +1, len(categorical)):
        gruped_product = dataset_merge.groupby([categorical[i],categorical[j]])[['ganado','counter']].cumsum()
        gruped_product[categorical[i] + '_'+ categorical[j] + '_mean']= np.NaN
        gruped_product.loc[(gruped_product['counter'] != 0), categorical[i] + '_'+ categorical[j] + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
        gruped_product[categorical[i] + '_'+ categorical[j] + '_mean'] = gruped_product['ganado']/ gruped_product['counter']
        gruped_product= gruped_product.rename(columns = {'ganado':(categorical[i] + '_'+ categorical[j] + '_ganados'), 'counter': (categorical[i] + '_'+ categorical[j] + '_count') })
        dataset_merge= dataset_merge.join(gruped_product)
        

In [ ]:
dataset.dtypes

# Convertir mondeas

In [11]:
dataset_merge['conversion'] = dataset_merge['ASP_(converted)']/dataset_merge['ASP']

cotizacion = dataset_merge.groupby(['ASP_Currency', 'Opportunity_Created_Date']).agg({'conversion': 'mean'}).reset_index()
#print(dataset[dataset['Stage']== 'Closed Won']['conversion'].value_counts())
cotizacion = cotizacion.rename(columns = {'conversion': 'conversion_diaria'})
cotizacion

dataset_merge= dataset_merge.merge(cotizacion, on = ['ASP_Currency', 'Opportunity_Created_Date'], how = 'left')

dataset_merge

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Product_Family_Month_count,Product_Family_Month_mean,Product_Family_Delivery_Quarter_ganados,Product_Family_Delivery_Quarter_count,Product_Family_Delivery_Quarter_mean,Month_Delivery_Quarter_ganados,Month_Delivery_Quarter_count,Month_Delivery_Quarter_mean,conversion,conversion_diaria
0,12714,EMEA,France,0,0,0,0,Bureaucratic_Code_4,2013-07-27,None,...,1,0.0,0,1,0.000000,0,1,0.000000,1.131079,1.131079
1,18684,APAC,Philippines,1,1,1,1,Bureaucratic_Code_3,2014-01-22,Source_3,...,1,0.0,0,1,0.000000,0,1,0.000000,1.000000,1.000000
2,20675,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,1,0.0,0,1,0.000000,0,1,0.000000,1.000000,1.000000
3,20682,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,1,0.0,0,2,0.000000,0,1,0.000000,1.000000,1.000000
4,20513,Americas,None,1,0,1,0,Bureaucratic_Code_5,2013-08-22,Source_7,...,1,1.0,1,1,1.000000,1,1,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19429,27867,EMEA,Belgium,1,1,1,1,Bureaucratic_Code_4,2015-06-17,Source_7,...,0,NaN,0,0,NaN,10,19,0.526316,1.131100,1.131100
19430,11318,Japan,None,1,1,0,0,Bureaucratic_Code_4,2019-02-12,None,...,0,NaN,0,0,NaN,0,5,0.000000,0.008990,0.008990
19431,5817,Japan,Japan,1,1,0,0,Bureaucratic_Code_4,2017-06-30,None,...,1,0.0,0,2,0.000000,10,19,0.526316,0.008990,0.008990
19432,16339,EMEA,KSA,1,1,0,0,Bureaucratic_Code_4,2017-06-12,Source_3,...,0,NaN,54,68,0.794118,0,2,0.000000,1.000000,1.000000


In [12]:
dataset_merge['conversion'] = dataset_merge.loc[:,'conversion'].fillna(dataset_merge['conversion_diaria'])

In [14]:
dataset_merge = dataset_merge.drop(dataset_merge[dataset_merge['conversion'] == np.inf].index)

In [16]:
dataset_merge.loc[dataset_merge['ASP_Currency'] == 'USD', 'conversion'] = 1
dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY' , 'conversion']= dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY' , 'conversion'].fillna( dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY', 'conversion'].mean())
dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR' , 'conversion']= dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR' , 'conversion'].fillna( dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR', 'conversion'].mean())
dataset_merge['Total_Amount_converted'] = dataset_merge['Total_Amount'] * dataset_merge['conversion']

# diferencias de tiempo

In [18]:
dataset_merge['delta_planed'] = dataset_merge['Planned_Delivery_End_Date'] - dataset_merge['Planned_Delivery_Start_Date']

In [19]:
dataset_merge.dtypes

ID                                              int64
Region                                         object
Territory                                      object
Pricing, Delivery_Terms_Quote_Appr              int64
Pricing, Delivery_Terms_Approved                int64
                                           ...       
Total_Amount_converted_median                 float64
Total_Amount_converted_std                    float64
Total_Amount_converted_max                    float64
Total_Amount_converted_min                    float64
delta_planed                          timedelta64[ns]
Length: 357, dtype: object

In [20]:
dataset_merge['acount_age'] = dataset_merge['Opportunity_Created_Date'] - dataset_merge['Account_Created_Date']

# features por oportunity ID

In [17]:
#Opportunity_ID
features= ['sum','mean','median','std', 'max', 'min']

dataset_numerico = dataset_merge[['Opportunity_ID','ASP', 'ASP_(converted)', 'TRF', 'Total_Amount', 'Total_Taxable_Amount', 'Total_Amount_converted']]
#ASP                                           float64
#ASP_(converted)                               float64
#TRF                                             int64
#Total_Amount                                  float64
#Total_Taxable_Amount                          float64
#Total_Amount_converted                        float64

features_num= dataset_numerico.groupby('Opportunity_ID').agg(features)
level_0 = features_num.columns.get_level_values(0)
level_1 = features_num.columns.get_level_values(1)
features_num.columns = level_0 + '_' + level_1
features_num.reset_index()
dataset_merge = dataset_merge.join(features_num, on= 'Opportunity_ID', how = 'left')

In [ ]:
dataset.dtypes

# elimino las columnas no propias

In [34]:
filter = dataset2['ID'].tolist()

In [24]:
dataset_merge.shape

(19433, 358)

In [28]:

dataset_merge = dataset_merge.drop(columns=['Region', 'Territory','Bureaucratic_Code','Account_Type', 'Billing_Country', 'Opportunity_Type',
                              'Account_Name','Account_Owner', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms',
                            'Pricing, Delivery_Terms_Quote_Appr',
                            'Sales_Contract_No', 'Opportunity_ID', 'Opportunity_Name','Source ', 'Account_Created_Date',
                            'Bureaucratic_Code_0_Approval', 'Bureaucratic_Code_0_Approved','Pricing, Delivery_Terms_Approved',
               'Last_Modified_By', 'Product_Family','Product_Name', 'Month', 'Delivery_Quarter','ganado','counter',
                            'Total_Taxable_Amount_Currency','Total_Taxable_Amount','Total_Amount_Currency','Total_Amount',
                           'TRF','Delivery_Year','Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'ASP_(converted)_Currency',
                           'ASP_(converted)', 'ASP_Currency', 'ASP', 'Quote_Expiry_Date', 'Last_Modified_Date','Opportunity_Created_Date'])

dataset_merge.shape

(19433, 316)

In [45]:
Test = dataset_merge.loc[dataset_merge['ID'].isin(filter)]
print(Test.shape)
Train= dataset_merge.loc[~dataset_merge['ID'].isin(filter)]
print(Train.shape)          

In [46]:
Test.to_csv('Features_Test_Lucas.csv')
Train.to_csv('Features_Train_Lucas.csv')